In [9]:
import ccxt
import pandas as pd
import math
from datetime import datetime, timedelta
import telegram_bot
import asyncio

#API 키 읽기
with open("../api(JKBOT).txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    secret = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key, #"XTpKrQGSk3GhXzqiEV4OfwGJzmTVcLh8dKGwHo4aQBH4p0mOqPDpIsxdh95tjGVf",
    'secret': secret, #"A0eqZGEWHsyL3NMM6WuDrucIanr7A2YZAnrwXVPhXpf2WGauIANwa5zsoNeNt0hs",
    'enableRateLimit' : True,
    'options': {
        'defaultType': 'future'
    }
})

today_date = ""

def ohlcv(symbol, limit=2000):
    btc_ohlcv = binance.fetch_ohlcv(symbol=symbol, timeframe='1d', since=None, limit=limit)
    df = pd.DataFrame(data=btc_ohlcv, columns=["datetime", "open", "high", "low", "close", "volumns"])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')


    return df
# def cal_amount(usdt_balance, cur_price, units):
#     amount = math.floor((usdt_balance * 1/units) / cur_price) / (1/units)
#     return amount
jongmok = []
markets = binance.load_markets() #마켈로드
balance = binance.fetch_balance() #계좌로드

# 선물종목 담아오기
# for market in markets.keys():
#     if market.endswith(":USDT"):
#         jongmok.append(market)

jongmok = ['BTC', 'ETH', 'STX', 'OP', 'XEM', 'MATIC', 'DYDX', 'SOL', 'APT', 'XRP', 'FIL', 'LDO', 'BNB', 'DOGE', 'AGIX', 'LTC', 'NEO', 'FTM', 'BNX', 'LINK', 'ANKR', 'CFX', 'ADA', 'GTC', 'GMT', 'FET', 'AVAX', 'ETC', '1000SHIB', 'DOT', 'GALA', 'SAND', 'STG', 'SSV', 'GRT', 'CRV', 'NEAR', 'EOS', 'YFI', 'APE', 'SNX', 'ACH', 'KLAY', 'MAGIC', 'MANA', 'AXS', 'BCH', 'MASK', 'FXS', 'OCEAN', 'HOOK', 'RNDR', 'ATOM', 'OMG', 'HIGH', 'AUDIO', 'DASH', 'AR', 'LINA', 'MKR', 'SUSHI', 'LIT', 'INJ', 'DUSK', 'TRX', 'BLZ', 'VET', 'KNC', 'REN', 'CHZ', 'IMX', 'THETA', 'AAVE', 'UNI', 'HBAR', 'ICP', 'WAVES', 'MINA', 'KAVA', 'ROSE', 'ZEC', 'ENS', 'QTUM', 'XTZ', 'COCOS', '1000LUNC', 'FLOW', 'ENJ', 'RLC', 'XLM', 'LRC', 'CELO', 'JASMY', 'IOTA', 'EGLD', 'UNFI', 'ONT', '1INCH', 'RUNE', 'ZIL', 'WOO', 'XMR', 'BEL', 'ALPHA', 'ICX', 'BAT', 'BAKE', 'ALGO', 'API3', 'C98', 'SXP', 'TRB', 'RSR', 'ONE', 'PEOPLE', 'COMP', 'PHB', 'STMX', 'SKL', 'LUNA2', 'KSM', 'STORJ', 'GAL', 'BAND', 'ALICE', 'ZRX', 'QNT', 'IOST', 'ZEN', 'CELR', 'MTL', 'REEF', 'COTI', 'ATA', 'HOT', 'FLM', 'DENT', 'IOTX', 'SFP', 'ASTR', 'RVN', 'CVX', 'ANT', 'TOMO', 'OGN', 'BAL', 'GMX', 'LPT', 'NKN', 'T', 'HNT', 'DAR', 'CTSI', 'DGB', 'CTK', 'ARPA', '1000XEC', 'SPELL', 'BTCDOM', 'FOOTBALL', 'DEFI', 'BLUEBIRD']


# 거래 최소단위

min_units = {}
for sym in jongmok:
    min_units[sym] = markets[sym+'/USDT']['limits']['market']['min']

#보유종목 조회
def account():
    global account_df
    print("계좌를 조회합니다.")
    balance = binance.fetch_balance()
    positions = balance['info']['positions'] # 포지션 조회
    df = pd.DataFrame(positions, columns = list(positions[0].keys()))
    account_df = df[df['initialMargin'] != "0"]
    position_tot = (pd.to_numeric(account_df['entryPrice']) * abs(pd.to_numeric(account_df['positionAmt']))).sum()
    cur_tot = pd.to_numeric(account_df['initialMargin']).sum()
    # unrealizedProfit = pd.to_numeric(account_df['unrealizedProfit']).sum()
    unrealizedProfit = balance['info']['assets'][6]['unrealizedProfit']
    marginBalance = balance['info']['assets'][6]['marginBalance'].split('.')[0].strip()
    print("보유금액: " +str(marginBalance) + " USDT", "매수금액: " + str(int(cur_tot)) + " USDT", "  미실현손익: " + str(int(round(float(unrealizedProfit)))) +" USDT")
    return account_df

# 현재가 조회
def cur_price(symbol):
    cur_price = binance.fetch_ticker(symbol+'/USDT')['last']
    # print(symbol +"의 현재가는 " + str(cur_price) + "입니다.")
    return cur_price
# 주문
def order(symbol, usdt, side):

    print(symbol + "을 " + str(usdt) + "USD "+ side +"합니다.")
    amount = math.floor((usdt/cur_price(symbol))/min_units[symbol])*min_units[symbol]
    # binance.create_market_order(symbol=symbol+'/USDT', amount=amount, side=side)

# 청산
# 청산
def close_order(symbol):
    print(symbol +"을 모두 청산합니다.")
    balance = binance.fetch_balance()
    positions = balance['info']['positions'] # 포지션 조회
    df = pd.DataFrame(positions, columns = list(positions[0].keys()))
    amount = float(df.loc[df.loc[:, 'symbol'] == symbol.replace("/USDT", "USDT")]['positionAmt'].iloc[0])
    if amount > 0:
        side = 'SELL'
    elif amount <0:
        side = 'BUY'
    binance.create_market_order(symbol=symbol+'/USDT', amount=abs(amount), side=side)

#거래내역 조회
def history():
    history= []
    print("거래내역을 조회합니다.")
    for sym in jongmok:
        d = binance.parse8601()
        fetchTrades = binance.fetch_my_trades(symbol=sym+'/USDT', since=d, limit=4000, params={'order': 'asc'})
        if len(fetchTrades) != 0:
            for k in range(len(fetchTrades)):
                fetchTrades[k]['fee'] = fetchTrades[k]['fee']['cost']
                fetchTrades[k]['realizedPnl'] = fetchTrades[k]['info']['realizedPnl']
                del fetchTrades[k]['info']
                del fetchTrades[k]['fees']
                history.append(fetchTrades[k])

    col = list(history[0].keys())
    col.append('realizedPnl')
    df = pd.DataFrame(history, columns=list(history[0].keys()))
    df['symbol'] = df['symbol'].str.slice(stop=-5)
    df['datetime'] = df['datetime'].str.slice(stop=-5)
    history_df = df.sort_values('datetime', ascending=True)
    history_df['realizedPnl'] = pd.to_numeric(history_df['realizedPnl'])
    history_df = history_df[['datetime', 'order', 'symbol', 'side', 'price', 'amount', 'cost', 'fee', 'realizedPnl']].reset_index(drop=True)
    history_df = history_df.groupby('order', as_index=False).agg({
        'datetime' : 'first',
        'order' : 'first',
        'symbol' : 'first',
        'side' : 'first',
        'price' : 'mean',
        'amount' : 'sum',
        'cost' : 'sum',
        'fee' : 'sum',
        'realizedPnl' : 'sum'
    })
    history_df = history_df.sort_values('datetime', ascending=True).reset_index(drop=True)
    print("조회하였습니다.")
    return history_df

#스캔
def end_order(symbol, amount, side):
    print(symbol + "을 " + str(amount) + " "+ side +"합니다.")
    # binance.create_market_order(symbol=symbol+'/USDT', amount=amount, side=side)

def scan():
    window = 28
    away = 7
    print("지표를 생성중입니다.")
    from tensorflow import keras
    import numpy as np
    model = keras.Sequential()
    model.add(keras.layers.LSTM(30, activation='relu', input_shape=(window, 4), dropout=0.3))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.load_weights('1d_28_7_(30).h5')
    global signal, ind_dict, ind_df
    x_data = {}
    ind_dict = {}
    ind = []
    ticker = []
    global today_date
    for symbol in jongmok[:80]:

        btc_ohlcv = binance.fetch_ohlcv(symbol=symbol+'/USDT', timeframe='1d', since=None, limit=window+1)
        df = pd.DataFrame(data=btc_ohlcv, columns=["datetime", "open", "high", "low", "close", "volumns"])
        df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')

        df.set_index('datetime', inplace=True)
        x_data[symbol] = df[['open', 'high', 'low', 'close']][:-1]

        ## 전처리
        mean = x_data[symbol].mean()
        std = x_data[symbol].std()
        x = (x_data[symbol] - mean) / std

        ticker.append(symbol)
        ind.append(model.predict(np.array([x]), verbose=0)[0][0].tolist())

    ind_df = pd.DataFrame([], columns=['symbol', 'ind', 'ascending'])
    ind_df['symbol']=ticker
    ind_df['ind']=ind
    ind_df['ascending'] = abs(ind_df['ind'] - 0.5)+0.5
    ind_df = ind_df.sort_values('ascending', ascending=False)
    print(str(df.iloc[-2].name)[:-9] + "기준으로 지표를 조회하였습니다.")
    today_date = df.index[-2]
    return ind_df

def signal(buy_std, sell_std):
    print("매수조건: " + str(buy_std) + " 매수조건: " + str(sell_std) + " 종목을 검색합니다.")
    global signal_list
    condition = (ind_df['ind'] > buy_std) | (ind_df['ind'] < sell_std)
    selected_rows = ind_df[condition]
    signal_list = selected_rows.values.tolist()
    return signal_list, ind_df


# 함수 정리
cur_price(symbol) 현재가 조회
order(symbol, usdt, side) 주문
close_order(symbol) 청산
account() 보유계좌
scan() 종목조회
signal(buy_std, sell_std) 매매신호

In [10]:
scan()

지표를 생성중입니다.
2023-03-04기준으로 지표를 조회하였습니다.


,symbol,ind,ascending
52,ATOM,0.964424,0.964424
45,AXS,0.956505,0.956505
57,AR,0.948426,0.948426
30,GALA,0.943666,0.943666
47,MASK,0.938395,0.938395
...,...,...,...
75,ICP,0.480978,0.519022
9,XRP,0.489986,0.510014
16,NEO,0.509247,0.509247
59,MKR,0.506117,0.506117


In [18]:
len(ind_df.loc[ind_df['ind'] > 0.5])/ (len(ind_df.loc[ind_df['ind'] < 0.5]) + len(ind_df.loc[ind_df['ind'] > 0.5]))


0.6125

In [9]:
ind_df['new_column'] = df.appl

,symbol,ind
52,ATOM,0.952895
137,IOTX,0.950345
100,WOO,0.943532
113,ONE,0.930205
114,PEOPLE,0.923871
...,...,...
138,SFP,0.114120
142,ANT,0.084718
110,SXP,0.075036
17,FTM,0.074993


In [8]:
account()

계좌를 조회합니다.
보유금액: 6350 USDT 매수금액: 3016 USDT   미실현손익: -146 USDT


,symbol,initialMargin,maintMargin,unrealizedProfit,positionInitialMargin,openOrderInitialMargin,leverage,isolated,entryPrice,maxNotional,positionSide,positionAmt,notional,isolatedWallet,updateTime,bidNotional,askNotional
19,TRXUSDT,195.12126000,1.26828819,-6.82294893,195.12126000,0,1,False,0.069611939614,3.0E7,BOTH,2901,195.12126000,0,1677561815448,0,0
27,DASHUSDT,184.14325881,1.84143258,12.27755956,184.14325881,0,1,False,72.58714648757,5000000.0,BOTH,-2.706,-184.14325881,0,1677766876147,0,0
36,SKLUSDT,191.97895116,3.83957902,-8.02944884,191.97895116,0,1,False,0.0439,3000000.0,BOTH,4556,191.97895116,0,1677888853978,0,0
53,RNDRUSDT,181.46422000,3.62928440,-18.58066000,181.46422000,0,1,False,1.3544,5000000.0,BOTH,147.7,181.46422000,0,1677888855599,0,0
67,HBARUSDT,585.16160319,5.85161603,-30.03546440,585.16160319,0,1,False,0.0681130585931,5000000.0,BOTH,9032,585.16160320,0,1677888854291,0,0
84,ARUSDT,368.40173852,3.68401738,-30.87956109,368.40173852,0,1,False,9.69129368932,5000000.0,BOTH,41.2,368.40173852,0,1677888854934,0,0
110,ASTRUSDT,166.40096741,3.32801934,-33.61681259,166.40096741,0,1,False,0.08174,5000000.0,BOTH,2447,166.40096741,0,1677455525551,0,0
140,LPTUSDT,158.06551321,1.58065513,-30.15917030,158.06551321,0,1,False,7.352526703499,5000000.0,BOTH,25.6,158.06551321,0,1677766897040,0,0
173,ANTUSDT,199.96860000,3.99937200,0.08060000,199.96860000,0,1,False,2.48,3000000.0,BOTH,80.6,199.96860000,0,1677888855286,0,0
197,FETUSDT,401.29830000,8.02596600,15.08672106,401.29830000,0,1,False,0.4491747800587,5000000.0,BOTH,-927,-401.29830000,0,1677767463475,0,0


In [31]:
'''청산'''
print("dashboard 파일을 불러옵니다.")
dashboard = pd.read_excel('dashboard.xlsx', index_col='날짜')

condition = (dashboard['만료일'] == today_date )
#
close_sym = dashboard[condition]
print(str(len(close_sym)) + "개의 종목을 청산합니다")
for i in range(len(close_sym)):
    if close_sym['side'][i] == 'SELL':
        side = 'BUY'
    if close_sym['side'][i] == 'BUY':
        side = 'SELL'
    end_order(close_sym['symbol'][i], close_sym['amount'][i], side)
    print(close_sym['symbol'][i] + "종목을" + str(close_sym['amount'][i]) + "개 청산하였습니다.")
# 만료일인 경우 statue를 open에서 close로 변경
dashboard.loc[condition, 'statue'] = 'close'

usdt = 200
scan()
signal(0.65, 0.35)
print("매매를 진행합니다.")
for i in range(len(signal_list)):
    # dashboard = pd.DataFrame('dashboard.xlsx')
    symbol = signal_list[i][0]
    if signal_list[i][1] > 0.5:
        side = 'BUY'
    if signal_list[i][1] < 0.5:
        side = 'SELL'
    amount = math.floor((usdt/cur_price(symbol))/min_units[symbol])*min_units[symbol]
    new_row = pd.DataFrame({
        'symbol' : [symbol], 'ind' : [signal_list[i][1]], 'side' : [side], 'amount' : [amount], 'statue' : ['open'], '만료일' : [today_date  + timedelta(days=7)]}, columns=dashboard.columns, index=[str(today_date)])
    dashboard = pd.concat([dashboard, new_row], ignore_index=False)
    order(signal_list[i][0], usdt, side)
dashboard.to_excel('dashboard.xlsx', index_label='날짜')
print("매매를 종료합니다")


dashboard 파일을 불러옵니다.
0개의 종목을 청산합니다
지표를 생성중입니다.
2023-03-01기준으로 지표를 조회하였습니다.
매수조건: 0.65 매수조건: 0.35 종목을 검색합니다.
매매를 진행합니다.
HBAR을 200USD BUY합니다.
LPT을 200USD BUY합니다.
FOOTBALL을 200USD BUY합니다.
ADA을 200USD BUY합니다.
LIT을 200USD BUY합니다.
ROSE을 200USD BUY합니다.
JASMY을 200USD BUY합니다.
IOTX을 200USD BUY합니다.
LDO을 200USD SELL합니다.
FET을 200USD SELL합니다.
DASH을 200USD SELL합니다.
매매를 종료합니다


In [33]:
account()

계좌를 조회합니다.
보유금액: 6435 USDT 매수금액: 1963 USDT   미실현손익: -63 USDT


,symbol,initialMargin,maintMargin,unrealizedProfit,positionInitialMargin,openOrderInitialMargin,leverage,isolated,entryPrice,maxNotional,positionSide,positionAmt,notional,isolatedWallet,updateTime,bidNotional,askNotional
19,TRXUSDT,200.64743292,1.30420831,-1.29677601,200.64743292,0,1,False,0.069611939614,3.0E7,BOTH,2901,200.64743292,0,1677561815448,0,0
27,DASHUSDT,193.82397081,1.93823970,2.59684756,193.82397081,0,1,False,72.58714648757,5000000.0,BOTH,-2.706,-193.82397081,0,1677766876147,0,0
67,HBARUSDT,401.67920000,4.01679200,-13.56404108,401.67920000,0,1,False,0.0698827406955,5000000.0,BOTH,5942,401.67920000,0,1677766844391,0,0
84,ARUSDT,186.30485406,1.86304854,-13.14114594,186.30485406,0,1,False,10.228,5000000.0,BOTH,19.5,186.30485406,0,1677455500858,0,0
110,ASTRUSDT,183.66389172,3.67327783,-16.35388828,183.66389172,0,1,False,0.08174,5000000.0,BOTH,2447,183.66389172,0,1677455525551,0,0
140,LPTUSDT,178.91840000,1.78918400,-9.30628352,178.91840000,0,1,False,7.352526703499,5000000.0,BOTH,25.6,178.91840000,0,1677766897040,0,0
197,FETUSDT,423.55940778,8.47118815,-7.17438672,423.55940778,0,1,False,0.4491747800587,5000000.0,BOTH,-927,-423.55940778,0,1677767463475,0,0
208,OCEANUSDT,195.08010000,3.90160200,-4.68350000,195.08010000,0,1,False,0.4052,5000000.0,BOTH,493,195.08010000,0,1677455461979,0,0


In [109]:
from binance.client import Client
client = Client(api_key, secret)

In [26]:
account()

계좌를 조회합니다.
보유금액: 6436 USDT 매수금액: 2150 USDT   미실현손익: -66 USDT


,symbol,initialMargin,maintMargin,unrealizedProfit,positionInitialMargin,openOrderInitialMargin,leverage,isolated,entryPrice,maxNotional,positionSide,positionAmt,notional,isolatedWallet,updateTime,bidNotional,askNotional
19,TRXUSDT,200.19801000,1.30128706,-1.74619893,200.19801000,0,1,False,0.069611939614,3.0E7,BOTH,2901,200.19801000,0,1677561815448,0,0
27,DASHUSDT,193.01265020,1.93012650,3.40816816,193.01265020,0,1,False,72.58714648757,5000000.0,BOTH,-2.706,-193.01265020,0,1677766876147,0,0
67,HBARUSDT,399.82820758,3.99828207,-15.41503350,399.82820758,0,1,False,0.0698827406955,5000000.0,BOTH,5942,399.82820758,0,1677766844391,0,0
84,ARUSDT,185.68499781,1.85684997,-13.76100219,185.68499781,0,1,False,10.228,5000000.0,BOTH,19.5,185.68499781,0,1677455500858,0,0
110,ASTRUSDT,182.52173000,3.65043460,-17.49605000,182.52173000,0,1,False,0.08174,5000000.0,BOTH,2447,182.52173000,0,1677455525551,0,0
140,LPTUSDT,177.90746060,1.77907460,-10.31722291,177.90746060,0,1,False,7.352526703499,5000000.0,BOTH,25.6,177.90746060,0,1677766897040,0,0
197,FETUSDT,617.48616908,12.34972338,-4.81176916,617.48616908,0,1,False,0.4491747800587,5000000.0,BOTH,-1364,-617.48616908,0,1677766392928,0,0
208,OCEANUSDT,194.24200000,3.88484000,-5.52160000,194.24200000,0,1,False,0.4052,5000000.0,BOTH,493,194.24200000,0,1677455461979,0,0


In [121]:
info = client.futures_account()
info

{'feeTier': 0,
 'canTrade': True,
 'canDeposit': True,
 'canWithdraw': True,
 'updateTime': 0,
 'multiAssetsMargin': False,
 'totalInitialMargin': '1335.49277126',
 'totalMaintMargin': '18.43141473',
 'totalWalletBalance': '6531.78987419',
 'totalUnrealizedProfit': '-58.39230989',
 'totalMarginBalance': '6473.39756430',
 'totalPositionInitialMargin': '1335.49277126',
 'totalOpenOrderInitialMargin': '0.00000000',
 'totalCrossWalletBalance': '6531.78987419',
 'totalCrossUnPnl': '-58.39230989',
 'availableBalance': '5137.44724447',
 'maxWithdrawAmount': '5137.44724447',
 'assets': [{'asset': 'DOT',
   'walletBalance': '0.00000000',
   'unrealizedProfit': '0.00000000',
   'marginBalance': '0.00000000',
   'maintMargin': '0.00000000',
   'initialMargin': '0.00000000',
   'positionInitialMargin': '0.00000000',
   'openOrderInitialMargin': '0.00000000',
   'maxWithdrawAmount': '0.00000000',
   'crossWalletBalance': '0.00000000',
   'crossUnPnl': '0.00000000',
   'availableBalance': '0.0000000

In [123]:
info = client.futures_account_trades()
info

[{'symbol': 'NEOUSDT',
  'id': 188895172,
  'orderId': 6078877780,
  'side': 'BUY',
  'price': '12.832',
  'qty': '75.69',
  'realizedPnl': '15.55643884',
  'marginAsset': 'USDT',
  'quoteQty': '971.25408',
  'commission': '0.38850163',
  'commissionAsset': 'USDT',
  'time': 1677179153496,
  'positionSide': 'BOTH',
  'buyer': True,
  'maker': False},
 {'symbol': 'NEOUSDT',
  'id': 188895173,
  'orderId': 6078877780,
  'side': 'BUY',
  'price': '12.832',
  'qty': '0.59',
  'realizedPnl': '0.12126171',
  'marginAsset': 'USDT',
  'quoteQty': '7.57088',
  'commission': '0.00302835',
  'commissionAsset': 'USDT',
  'time': 1677179153496,
  'positionSide': 'BOTH',
  'buyer': True,
  'maker': False},
 {'symbol': 'NEOUSDT',
  'id': 188895174,
  'orderId': 6078877780,
  'side': 'BUY',
  'price': '12.832',
  'qty': '0.51',
  'realizedPnl': '0.10481944',
  'marginAsset': 'USDT',
  'quoteQty': '6.54432',
  'commission': '0.00261772',
  'commissionAsset': 'USDT',
  'time': 1677179153496,
  'position

NameError: name 'self' is not defined